In [3]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.2 MB/s eta 0:00:00


In [11]:
import logging
logging.getLogger("praw").setLevel(logging.ERROR)

In [12]:
import praw
import pandas as pd
import re

reddit = praw.Reddit(
    client_id="Ddx2_gIABdN4rS3cbHzfRw",
    client_secret="M9lYH7DDd6e5DS0erQ1by-MPFArSew",
    user_agent="behavioral-analysis"
)

keywords= ["I hate myself", "tired of life","no reason to live","can't stop drinking","giving up","relapsed","drug problem","someone talk to me","addiction help","ending it","self harm","suicidal","I can't take it anymore"]
subreddits = ["depression", "Anxiety", "SuicideWatch", "addiction", "mentalhealth", "self","BPD","depression_help","stopdrinking","offmychest"]

EXCLUDE_PHRASES = [
    "I apologize", "If you're struggling", "Here are some resources",
    "call this number", "for more information", "I'm sorry", "take care",
    "reach out", "helpline", "contact support", "resources for help"
]

def clean_text(text):
    if not text:
        return None
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", "", text)
    text = text.lower()
    return text

def valid_post(content):
    if not any(keyword.lower() in content for keyword in keywords):
        return False
    if any(phrase.lower() in content for phrase in EXCLUDE_PHRASES):
        return False
    if len(content.split()) < 10:
        return False
    return True

def fetch_reddit_posts(subreddits, limit=3000):
    data = []
    for subreddit in subreddits:
        print(f"Scraping r/{subreddit}...")
        for post in reddit.subreddit(subreddit).new(limit=limit):
            content = f"{post.title} {post.selftext}"
            cleaned_content = clean_text(content)


            if cleaned_content and valid_post(cleaned_content):
                data.append({
                    "post_id": post.id,
                    "subreddit": subreddit,
                    "title": post.title,
                    "content": post.selftext,
                    "upvotes": post.score,
                    "comments": post.num_comments,
                    "created_at": post.created_utc
                })
    return pd.DataFrame(data)
reddit_df = fetch_reddit_posts(subreddits)
reddit_df.to_csv("filtered_reddit_posts.csv", index=False)

print(f"Saved {len(reddit_df)} filtered posts to filtered_reddit_posts.csv")
print(reddit_df.head())

Scraping r/depression...
Scraping r/Anxiety...
Scraping r/SuicideWatch...
Scraping r/addiction...
Scraping r/mentalhealth...
Scraping r/self...
Scraping r/BPD...
Scraping r/depression_help...
Scraping r/stopdrinking...
Scraping r/offmychest...
Saved 850 filtered posts to filtered_reddit_posts.csv
   post_id   subreddit                                              title  \
0  1j7rxfd  depression              I miss my parents despite their abuse   
1  1j7plai  depression  its getting really bad. im non functional, los...   
2  1j7pcmk  depression             Can someone please say something kind?   
3  1j7ocqy  depression           Death seems the only reasonable solution   
4  1j7o7ft  depression                                  I wish I was dead   

                                             content  upvotes  comments  \
0  This past month I really realized I have no fa...        1         2   
1  im 22m and completely going off the rails. dep...        6         2   
2    \nHi, I'm